In [1]:
from cellpose import models, io
from cellpose.io import *
from collections import defaultdict
import geopandas
import glob
import imagej
from jpype import JArray, JInt
import matplotlib.pyplot as plt
import multiprocessing as mp
import numpy as np
import os
import re
import pandas
from pandas import DataFrame
from pathlib import Path
import scyjava
import seaborn
import shutil
import tkinter as tk
from tkinter import filedialog
from PIL import Image
import sys
import os

In [ ]:
scyjava.config.add_option('-Xmx30g')
#ij = imagej.init('/home/saka/fiji-linux64/Fiji.app', mode = 'interactive')
ij = imagej.init('/home/saka/sw/local/fiji/2023', mode='interactive')
ij.ui().showUI()
ij.getVersion()

In [3]:
showPolygonRoi = scyjava.jimport('ij.gui.PolygonRoi')
Overlay = scyjava.jimport('ij.gui.Overlay')
Regions = scyjava.jimport('net.imglib2.roi.Regions')
LabelRegions = scyjava.jimport('net.imglib2.roi.labeling.LabelRegions')
ZProjector = scyjava.jimport('ij.plugin.ZProjector')()
Duplicator = scyjava.jimport('ij.plugin.Duplicator')()
ov = Overlay()
Model =  scyjava.jimport('fiji.plugin.trackmate.Model')
Settings= scyjava.jimport('fiji.plugin.trackmate.Settings')
TrackMate = scyjava.jimport('fiji.plugin.trackmate.TrackMate')
Settings= scyjava.jimport('fiji.plugin.trackmate.Settings')
TrackMate = scyjava.jimport('fiji.plugin.trackmate.TrackMate')
Logger= scyjava.jimport('fiji.plugin.trackmate.Logger')
DetectorKeys= scyjava.jimport('fiji.plugin.trackmate.detection.DetectorKeys') 
ExportTracksToXML= scyjava.jimport('fiji.plugin.trackmate.action.ExportTracksToXML') 
TmXmlWriter= scyjava.jimport('fiji.plugin.trackmate.io.TmXmlWriter')
LogRecorder = scyjava.jimport('fiji.plugin.trackmate.util.LogRecorder')
SparseLAPTrackerFactory= scyjava.jimport('fiji.plugin.trackmate.tracking.jaqaman.SparseLAPTrackerFactory')
TMUtils = scyjava.jimport('fiji.plugin.trackmate.util.TMUtils')
HyperStackDisplayer = scyjava.jimport('fiji.plugin.trackmate.visualization.hyperstack.HyperStackDisplayer')
SelectionModel = scyjava.jimport('fiji.plugin.trackmate.SelectionModel')
CellposeDetectorFactory = scyjava.jimport('fiji.plugin.trackmate.cellpose.CellposeDetectorFactory')
FeatureFilter = scyjava.jimport('fiji.plugin.trackmate.features.FeatureFilter')
DisplaySetting = scyjava.jimport('fiji.plugin.trackmate.gui.displaysettings.DisplaySettings')
DisplaySettingsIO = scyjava.jimport('fiji.plugin.trackmate.gui.displaysettings.DisplaySettingsIO')
CaptureOverlayAction = scyjava.jimport('fiji.plugin.trackmate.action.CaptureOverlayAction')
PretrainedModel= scyjava.jimport('fiji.plugin.trackmate.cellpose.CellposeSettings.PretrainedModel')
ThresholdDetectorFactory= scyjava.jimport('fiji.plugin.trackmate.detection.ThresholdDetectorFactory')
TrackScheme = scyjava.jimport('fiji.plugin.trackmate.visualization.trackscheme.TrackScheme')
TrackTableView = scyjava.jimport('fiji.plugin.trackmate.visualization.table.TrackTableView')
AllSpotsTableView = scyjava.jimport('fiji.plugin.trackmate.visualization.table.AllSpotsTableView')

rm = ij.RoiManager.getRoiManager()

### Attempt to make functions for the workflow and future package

In [ ]:
def process_images(input_path, output_directory, method='max all', wanted_z=2):
    """
    This function aim to create two images from the raw image, a multi-channel time-lapse of a single z as trackmate-cellpose input, 
    and a multi-channel z-projection of the time lapse for a better visual base for single cell selection (also lighter for the RAM...)
    """
    
    # Load raw image
    raw_image = ij.io().open(input_path)
    imp = ij.py.to_imageplus(raw_image)
    
    # Creation of single slice T-stack
    channels = raw_image[:, :, :, wanted_z, :]
    final_image = ij.py.to_imageplus(channels)
    final_image.setDimensions(4, 1, 121)
    channels_image = output_directory + "/channels.tif"
    saved = ij.IJ.save(final_image, ij.py.to_java(channels_image))
    
    # Creation of 'max' z-projection of raw image for future selection of cells of interest
    zproject = ZProjector.run(imp, method)
    zproject.setDimensions(4, 1, 121)
    max_channels = output_directory + "/max_channels.tif"
    ij.IJ.saveAs(zproject, "Tiff", ij.py.to_java(max_channels))

    return channels_image, max_channels


def process_channels(LC3_channel = 0, bact_channel = 1, LV_channel = 2, format = "Tiff"):
    """
    this function is to isolate the channels into distinct z-stack time lapse
    """
    
    channel = raw_image[:, :, LC3_channel, :, :]
    lc3 = ij.py.to_imageplus(channel)
    lc3.setDimensions(1, 11, 121)
    #ij.ui().show(lc3)
    LC3_image = f"{directory_path}/channel_LC3.tif"
    ij.IJ.saveAs(lc3, format, ij.py.to_java(LC3_image))

    channel = raw_image[:, :, LV_channel, :, :]
    lv = ij.py.to_imageplus(channel)
    lv.setDimensions(1, 11, 121)
    LV_image = f"{directory_path}/channel_LV.tif"
    ij.IJ.saveAs(lv, "Tiff", ij.py.to_java(LV_image))

    channel = raw_image[:, :, bact_channel, :, :]
    bact = ij.py.to_imageplus(channel)
    bact.setDimensions(1, 11, 121)
    #ij.ui().show(bact)
    bact_image = f"{directory_path}/channel_bact.tif"
    ij.IJ.saveAs(bact, "Tiff", ij.py.to_java(bact_image))

## Cellpose to ROI

In [ ]:
base_image = Path(f"/home/saka/Documents/Lab_stuff/confocal/exp2/stack.tif").as_posix()
open_image = f'open("{base_image}")'
imp = ij.py.run_macro(open_image)
ij.ui().show(imp)

In [ ]:
data_info = {}
for element in range(len(imp.dims)):
    name = imp.dims[element]
    data_info[name] = imp.shape[element]
    print(name)
num_frame = data_info['T'] + 1

In [ ]:
for timepoint in range(1, num_frame):
    frame_number = timepoint - 1
    input_txt = Path(f"{directory_path}/outputs/Experiment-1647-Split Scenes-01_z2/cellpose/frame_{frame_number}_cp_outlines.txt")
    txt_fh = open(input_txt, 'r')
    for line in txt_fh:
            xy = line.rstrip().split(",")
            xy_coords = [int(element) for element in xy if element not in '']
            x_coords = [int(element) for element in xy[::2] if element not in '']
            y_coords = [int(element) for element in xy[1::2] if element not in '']
            xcoords_jint = JArray(JInt)(x_coords)
            ycoords_jint = JArray(JInt)(y_coords)
            polygon_roi_instance = scyjava.jimport('ij.gui.PolygonRoi')
            roi_instance = scyjava.jimport('ij.gui.Roi')
            imported_polygon = polygon_roi_instance(xcoords_jint, ycoords_jint, len(x_coords), int(roi_instance.POLYGON))
            imp.setRoi(imported_polygon)
            rm.addRoi(imported_polygon)
            roi_count = rm.getCount()
            rm.select(roi_count - 1)
            time_set = imp.setT(timepoint)   
            rm.runCommand('Update')
ij.py.run_macro("roiManager('Select All');")
f_name = os.path.basename(filepath)
f_name = os.path.splitext(f_name)[0]
rm.runCommand("Save", f"{directory_path}/" + f"{f_name}.zip")

# workflow 2.0

In [4]:
#define your parent directory
directory_path = '/home/saka/Documents/Lab_stuff/confocal/exp1ATG7_scene3'
bact_path = directory_path + "/bact/"
if not os.path.exists(bact_path):
    os.makedirs(bact_path)
other_path = directory_path + "/other/"
if not os.path.exists(other_path):
    os.makedirs(other_path)

In [ ]:
# open raw file
raw_image = ij.io().open('/home/saka/Documents/Lab_stuff/confocal/exp1ATG7_scene3/Experiment-1647-Split-Scenes-03.czi')

### Images needed for single cell tracking
The T-stack is used for cell detection with trackmate. The 'max' z-projection is used to select cells that would need to be infected but not overloaded for our future single bacteria analysis. 

In [ ]:
#creation of single slice T-stack
wanted_z = 2 # change this number based on your original cellpose training dataset
channels = raw_image[:, :, :, wanted_z, :]
final_image = ij.py.to_imageplus(channels)
final_image.setDimensions(4, 1, 121) # this is to make sure the dimensions are not mixed up
channels_image = directory_path + "/channels.tif"
saved = ij.IJ.save(final_image, ij.py.to_java(channels_image))

# creation of 'max' z-projection of raw image for future selection of cells of interest
method = 'max all'
zproject = ZProjector.run(imp, method)
zproject.setDimensions(4, 1, 121)
max_channels = directory_path + "/max_channels.tif"
ij.IJ.saveAs(zproject, "Tiff", ij.py.to_java(max_channels))

In [ ]:
imp = ij.py.to_imageplus(raw_image)
method = 'max all'
zproject = ZProjector.run(imp, method)
zproject.setDimensions(4, 1, 121)
max_channels = directory_path + "/max_channels.tif"
ij.IJ.saveAs(zproject, "Tiff", ij.py.to_java(max_channels))

In [ ]:
# Usage example of process_image function
raw_image_path = '/home/saka/Documents/Lab_stuff/confocal/exp1ATG7_scene3/Experiment-1647-Split-Scenes-03.czi'
directory_path = '/home/saka/Documents/Lab_stuff/confocal/exp1ATG7_scene3'

channels, max_channels = process_images(raw_image_path, directory_path, method='max all', wanted_z=2)

### single channel z stack isolation and saving

In [ ]:
format = f'Tiff'

LC3_channel = 0
channel = raw_image[:, :, LC3_channel, :, :]
lc3 = ij.py.to_imageplus(channel)
lc3.setDimensions(1, 11, 121)
#ij.ui().show(lc3)
result_path = f"{directory_path}/channel_LC3.tif"
ij.IJ.saveAs(lc3, "Tiff", ij.py.to_java(result_path))

LV_channel = 2
channel = raw_image[:, :, LV_channel, :, :]
lv = ij.py.to_imageplus(channel)
lv.setDimensions(1, 11, 121)
result_path = f"{directory_path}/channel_LV.tif"
ij.IJ.saveAs(lv, "Tiff", ij.py.to_java(result_path))

bact_channel = 1
channel = raw_image[:, :, bact_channel, :, :]
bact = ij.py.to_imageplus(channel)
bact.setDimensions(1, 11, 121)
#ij.ui().show(bact)
result_path = f"{directory_path}/channel_bact.tif"
ij.IJ.saveAs(bact, "Tiff", ij.py.to_java(result_path))

### single cell tracking with Trackmate-Cellpose

In [ ]:
image = "/home/saka/Documents/Lab_stuff/confocal/exp1ATG7_scene3/cell_channel.tif"
out = "/home/saka/Documents/Lab_stuff/confocal/exp1ATG7_scene3/trackmate/"
if not os.path.exists(out):
    os.makedirs(out)
    
# Parameters for Detection
# Here, the user can specify parameters for the detection step in Trackmate (Threshold Detector)
dsettings = {
    'TARGET_CHANNEL' : ij.py.to_java(0),
    'SIMPLIFY_CONTOURS' : True,
    'CELLPOSE_MODEL' : PretrainedModel.CUSTOM,
    'CELLPOSE_MODEL_FILEPATH' : "/home/saka/Documents/CP_20220523_104016",
    'USE_GPU' : True,
    'OPTIONAL_CHANNEL_2' : ij.py.to_java(0),
    'CELL_DIAMETER' : 16.38,
    'CELLPOSE_PYTHON_FILEPATH' : "/home/saka/sw/local/fiji/2023/bin/python3.10",
}

# Parameters for Tracking
# Here, the user can specify parameters for the tracking step in Trackmate (LAP Tracker)
frame_gap = 2
tsettings = {
    'LINKING_MAX_DISTANCE' : 10.0,
    'ALLOW_GAP_CLOSING' : True,
    'GAP_CLOSING_MAX_DISTANCE' : 15.0,
    'MAX_FRAME_GAP' : ij.py.to_java(2),
    'ALLOW_TRACK_SPLITTING' : False,
    'SPLITTING_MAX_DISTANCE' : 15.0,
    'ALLOW_TRACK_MERGING' : False,
    'ALTERNATIVE_LINKING_COST_FACTOR' : 1.05,
    'MERGING_MAX_DISTANCE' : 15.0,
    'CUTOFF_PERCENTILE' : 0.9,
}

if (image[len(image)-4:] == ".tif"):

    # Open Image
    imp = ij.IJ.openImage(image)
    imp.show()
    ij.py.run_macro('run("Set Scale...", "distance=2048 known=202.83 unit=µm");')
            
    # Create Model
    model = Model()
    settings = Settings(imp)
        
    # Detector
    settings.detectorFactory = CellposeDetectorFactory()
    for parameter, value in dsettings.items():
        settings.detectorSettings[parameter] = value
        
    # Tracker
    settings.trackerFactory = SparseLAPTrackerFactory()
    settings.trackerSettings = settings.trackerFactory.getDefaultSettings()
    for parameter, value in tsettings.items():
        settings.trackerSettings[parameter] = value
        
    # Execute Tracking
    trackmate = TrackMate(model, settings)
    ok = trackmate.checkInput()
    if not ok:
        sys.exit(str(trackmate.getErrorMessage()))
    ok = trackmate.process()
    if not ok:
        sys.exit(str(trackmate.getErrorMessage()))
    selectionModel = SelectionModel(model)
        
    # Display
    ds = DisplaySettingsIO.readUserDefault()
    displayer = HyperStackDisplayer(model, selectionModel, imp, ds)
    displayer.render()
    displayer.refresh()
    trackscheme = TrackScheme(model, selectionModel, ds)
    trackscheme.render()
        
    # Save Data
    outFile = Path(out+"exportModel.xml")
    writer = TmXmlWriter(outFile)
    writer.appendModel(model)
    writer.appendSettings(settings)
    writer.writeToFile()
    csvFileTracks = Path(out+"exportTracks.csv")
    trackTableView = TrackTableView(model, selectionModel, ds)
    trackTableView.getSpotTable().exportToCsv(csvFileTracks)
    

# sort csv file from trackmate output

In [ ]:
df = pandas.read_csv("/home/saka/Documents/Lab_stuff/confocal/exp1ATG7_scene3/trackmate/exportTracks.csv")
df.drop([0, 1, 2], axis=0, inplace=True)
df = df.reset_index(drop=True)

for index, row in df.iterrows():
    df.at[index, 'TRACK_ID'] = int(row['TRACK_ID'])
    df.at[index, 'FRAME'] = int(row['FRAME'])

df.sort_values(by=['TRACK_ID', 'FRAME'], inplace=True, ignore_index=True)
df.to_csv("/home/saka/Documents/Lab_stuff/confocal/exp1ATG7_scene3/trackmate/exportTracks_edited.csv", index=False)

# obtain ROI set

In [ ]:
IJRoiExporter = scyjava.jimport('fiji.plugin.trackmate.action.IJRoiExporter')
model.setLogger(Logger.IJ_LOGGER)
logger = Logger.IJ_LOGGER
exporter = IJRoiExporter(imp, logger)
exporter.export(model.getSpots().iterable(True))
rm = ij.RoiManager.getRoiManager()
rm.save("/home/saka/Documents/Lab_stuff/confocal/exp1ATG7_scene3/RoiSet_cells.zip");
rm.runCommand(imp,"Delete");

In [ ]:
from zipfile import ZipFile
with ZipFile("/home/saka/Documents/Lab_stuff/confocal/exp1ATG7_scene3/RoiSet_cells.zip", 'r') as zip: 
    zip.extractall(path="/home/saka/Documents/Lab_stuff/confocal/exp1ATG7_scene3/rois")

# create csv of tracks by row

In [ ]:
data = pandas.read_csv("/home/saka/Documents/Lab_stuff/confocal/exp1ATG7_scene3/trackmate/exportTracks_edited.csv")
print(data)

import csv
file = open('/home/saka/Documents/Lab_stuff/confocal/exp1ATG7_scene3/trackmate/da_Tracks.csv', 'w', newline='')
csv_writer = csv.writer(file)

current_track = 0
first = True
string = ''
for index, row in data.iterrows():
    if int(row['TRACK_ID']) == current_track:
        if first:
            string = f'{index+1}'
            first = False
        else:
            string = string + ", " + f'{index+1}'
    else:
        csv_writer.writerow([string])
        print(string)
        print(current_track)

        string = f'{index+1}'
        current_track += 1
file.close()

# add ROIs to ROI Manager

In [10]:
df = pandas.read_csv("/home/saka/Documents/Lab_stuff/confocal/exp1ATG7_scene3/trackmate/exportTracks_edited.csv")
roiset = "/home/saka/Documents/Lab_stuff/confocal/exp1ATG7_scene3/rois/"
for index, row in df.iterrows():
    file = row['LABEL'] + '.roi'
    #imp2 = ij.IJ.openImage(roiset+file)
    rm.open(roiset+file)

# add color overlay for each track

In [ ]:
max_channels = directory_path + "/max_channels.tif"
imp = ij.IJ.openImage(max_channels)
imp.show()

In [ ]:
data_info = {}
for element in range(len(imp.dims)):
    name = imp.dims[element]
    data_info[name] = imp.shape[element]
    print(name)
num_channel = data_info['C'] + 1

In [ ]:
for n in range (1, num_channel):
    imp.setC(n)
    ij.IJ.resetMinAndMax(imp)
imp.setC(1)
ij.IJ.run(imp, "Green", "");
imp.setC(2)
ij.IJ.run(imp, "Red", "");

In [ ]:
df2 = pandas.read_csv("/home/saka/Documents/Lab_stuff/confocal/exp1ATG7_scene3/trackmate/da_Tracks.csv", header=None)

#nearest_path = "C:/Users/volkenlab/Documents/Anushka/nearest.csv"
#testing = pandas.read_csv(nearest_path)
rm = ij.RoiManager.getRoiManager()

import random
colors = ["black", "blue", "cyan", "green", "magenta", "orange", "red", "white", "yellow"]

for index, row in df2.iterrows():
    l = row[0].split(", ")
    print(l)
    random_color = random.choice(colors)
    for cell in l:
        cell_index = int(cell)
        roi = rm.select(cell_index -1)
        overlay_command = f"Overlay.addSelection('{random_color}', 5);"
        ij.py.run_macro(overlay_command)

### selection of single cells and quality control

In [ ]:
cell1 = [91, 3]
cell2 = [9]
cell3 = [10]

In [ ]:
# testing the merging of 2 tracks
part1 = df2.iloc[91]
part1 = part1[0].split(", ")
part2 = df2.iloc[3]
part2 = part2[0].split(", ")
final_part = part1 + part2
#print(final_part)

for roi in final_part: #just for quality control to overlay the cell with a single color
    cellid = int(roi)
    rm.select(cellid - 1)
    overlay_command = f"Overlay.addSelection('Red', 20);"
    ij.py.run_macro(overlay_command)

In [ ]:
cell = df2.iloc[10]
cell2 = cell[0].split(", ")
for roi in cell2: 
    cellid = int(roi)
    rm.select(cellid - 1)
    overlay_command = f"Overlay.addSelection('Green', 20);"
    ij.py.run_macro(overlay_command)

## Test Area

In [ ]:
rm = ij.RoiManager.getRoiManager()
directory_path = '/home/saka/Documents/Lab_stuff/confocal/exp1ATG7_scene3'

### Duplication of single cell signal 

In [ ]:
cells_desired = [19, 14, 113, 45, 5, [32, 3], [18, 68], [16, 128], [40, 14]]

In [13]:
# assumes that "add ROIs to ROI Manager" has already been run

cells_desired = [19, 14, 113, 45, 5, [32, 3], [18, 68], [16, 128], [40, 14]] ## here the user can specify which row of Da_Tracks they want to obtain a single cell video for (from excel)
signal_list = ["bact", "LC3", "LV"] ## here the user can specify which signal videos they want to isolate the cell from
for signal in signal_list:
    image_path = Path(f"{directory_path}/channel_{signal}.tif").as_posix()
    imp = ij.IJ.openImage(image_path)
    ij.ui().show(imp)
    w = scyjava.jimport('ij.WindowManager')
    df = pandas.read_csv(directory_path + "/trackmate/da_Tracks.csv", header=None)
    for cell_desired in cells_desired:
        nums = []
        if isinstance(cell_desired, int):
            row = df.iloc[cell_desired-1]
            nums = row[0].split(", ")
        elif isinstance(cell_desired, list):
            for c in cell_desired:
                row = df.iloc[c-1]
                nums += row[0].split(", ")
        arr = []
        for x in nums:
            arr.append(int(x))
        rm = ij.RoiManager.getRoiManager()
        for i in range(len(arr)):
            cell = arr[i]
            roi = rm.getRoi(cell-1)
            roi2 = rm.select(cell-1)
            frame = roi.getTPosition()
            if i == 0:
                ij.IJ.run("Duplicate...", "title=Stack_reg_image duplicate frames="+str(frame))
                ij.IJ.run("Clear Outside", "stack")
                ij.IJ.selectWindow(imp.getTitle())
            elif i > 0:
                ij.IJ.run("Duplicate...", "title=channels duplicate frames="+str(frame))
                ij.IJ.run("Clear Outside", "stack")
                ij.IJ.run("Concatenate...", "title=Stack_reg_image open image1=Stack_reg_image image2=channels image3=[-- None --]")
                ij.IJ.selectWindow(imp.getTitle())
        new_vid =  w.getImage("Stack_reg_image")
        if signal == "bact":
            ij.IJ.save(new_vid, directory_path+"/bact/"+"cell_"+str(cell_desired)+".tif")
        else:
            ij.IJ.save(new_vid, directory_path+"/other/"+signal+"_cell_"+str(cell_desired)+".tif")
        w.getImage("Stack_reg_image").close()
        ij.IJ.selectWindow(imp.getTitle())
    ij.py.run_macro('close("*")')

In [ ]:
import tkinter as tk
from tkinter import filedialog
root = tk.Tk()
root.withdraw()
directory_path = filedialog.askdirectory()

### Bacterial ROI collection and coordinates measurement

In [ ]:
rm = ij.RoiManager.getRoiManager()
file_pattern = os.path.join(bact_path, "*.tif")
file_list = glob.glob(file_pattern)
for file_path in file_list :
    image = ij.io().open(file_path)
    shown = ij.ui().show(image)
    roi_collection = """
    run("Duplicate...", "duplicate");
    run("Smooth", "stack");
    run("Smooth", "stack");
    setAutoThreshold("Default dark no-reset");
    run("Threshold...");
    setThreshold(5, 255);
    setOption("BlackBackground", true);
    run("Convert to Mask", "black");  
    run("Analyze Particles...", "size=1-Infinity add stack");
    run("Set Measurements...", "area centroid stack redirect=None decimal=2");
    nbArea=roiManager("count")
    for (i=0; i<nbArea; i++) {
		roiManager("Select", i);
		run("Measure");	
    }
    close();
    close();
    """
    rois = ij.py.run_macro(roi_collection)
    f_name = os.path.basename(file_path)
    f_name = os.path.splitext(f_name)[0]
    rm.runCommand("Select All")
    rm.runCommand("Save", f"{bact_path}/" + f"{f_name}.zip") # this saves the ROIs as a zip file
    rm.runCommand("Delete")
    measurements = ij.ResultsTable.getResultsTable() # call of the table
    measurements_table = ij.convert().convert(measurements, scyjava.jimport('org.scijava.table.Table')) # conversion to a java table object
    table = ij.py.from_java(measurements_table) # Conversion into a python dataframe from Java
    output_path = Path(f"{bact_path}/{f_name}.csv") # save giving a name matching the opened image
    table.to_csv(output_path)
    ij.py.run_macro(""" 
        title = Table.title();
        selectWindow(title);
        run("Close");
    """)    

In [ ]:
bact_path = directory_path + "/bact/"
file_pattern = os.path.join(bact_path, "*.tif")
file_list = glob.glob(file_pattern)

In [ ]:
print(file_list)

### running trackmate on bacteria channel

In [ ]:
import sys
import os

Model =  scyjava.jimport('fiji.plugin.trackmate.Model')
Settings= scyjava.jimport('fiji.plugin.trackmate.Settings')
TrackMate = scyjava.jimport('fiji.plugin.trackmate.TrackMate')
Logger= scyjava.jimport('fiji.plugin.trackmate.Logger')
DetectorKeys= scyjava.jimport('fiji.plugin.trackmate.detection.DetectorKeys') 
ExportTracksToXML= scyjava.jimport('fiji.plugin.trackmate.action.ExportTracksToXML') 
TmXmlWriter= scyjava.jimport('fiji.plugin.trackmate.io.TmXmlWriter')
LogRecorder = scyjava.jimport('fiji.plugin.trackmate.util.LogRecorder')
SparseLAPTrackerFactory= scyjava.jimport('fiji.plugin.trackmate.tracking.jaqaman.SparseLAPTrackerFactory')
TMUtils = scyjava.jimport('fiji.plugin.trackmate.util.TMUtils')
HyperStackDisplayer = scyjava.jimport('fiji.plugin.trackmate.visualization.hyperstack.HyperStackDisplayer')
SelectionModel = scyjava.jimport('fiji.plugin.trackmate.SelectionModel')
CellposeDetectorFactory = scyjava.jimport('fiji.plugin.trackmate.cellpose.CellposeDetectorFactory')
FeatureFilter = scyjava.jimport('fiji.plugin.trackmate.features.FeatureFilter')
DisplaySetting = scyjava.jimport('fiji.plugin.trackmate.gui.displaysettings.DisplaySettings')
DisplaySettingsIO = scyjava.jimport('fiji.plugin.trackmate.gui.displaysettings.DisplaySettingsIO')
CaptureOverlayAction = scyjava.jimport('fiji.plugin.trackmate.action.CaptureOverlayAction')
PretrainedModel= scyjava.jimport('fiji.plugin.trackmate.cellpose.CellposeSettings.PretrainedModel')
ThresholdDetectorFactory= scyjava.jimport('fiji.plugin.trackmate.detection.ThresholdDetectorFactory')
TrackScheme = scyjava.jimport('fiji.plugin.trackmate.visualization.trackscheme.TrackScheme')
TrackTableView = scyjava.jimport('fiji.plugin.trackmate.visualization.table.TrackTableView')
AllSpotsTableView = scyjava.jimport('fiji.plugin.trackmate.visualization.table.AllSpotsTableView')

#reload(sys)
#sys.setdefaultencoding("utf-8")

# Directory
# Here, the user can specify the directory where the cell images are located
src = bact_path
out = src+"Output/"
if not os.path.exists(out):
    os.makedirs(out)

# Parameters for Detection
# Here, the user can specify parameters for the detection step in Trackmate (Threshold Detector)
dsettings = {
    'TARGET_CHANNEL' : ij.py.to_java(1),
    'SIMPLIFY_CONTOURS' : False,
    'INTENSITY_THRESHOLD' : 5.0,
}

# Parameters for Tracking
# Here, the user can specify parameters for the tracking step in Trackmate (LAP Tracker)
frame_gap = 2
tsettings = {
    'LINKING_MAX_DISTANCE' : 45.0,
    'ALLOW_GAP_CLOSING' : True,
    'GAP_CLOSING_MAX_DISTANCE' : 45.0,
    'MAX_FRAME_GAP' : ij.py.to_java(2),
    'ALLOW_TRACK_SPLITTING' : False,
    'SPLITTING_MAX_DISTANCE' : 15.0,
    'ALLOW_TRACK_MERGING' : False,
}

for image in os.listdir(src):
    if (image[len(image)-4:] == ".tif"):

        # Open Image
        imp = ij.IJ.openImage(src + image)
        imp.show()
        ij.IJ.run("Smooth", "stack")
                
        # Create Model
        model = Model()
        settings = Settings(imp)
        
        # Detector
        settings.detectorFactory = ThresholdDetectorFactory()
        for parameter, value in dsettings.items():
            #settings.detectorSettings.update({parameter:value})
            settings.detectorSettings[parameter] = value
        filter1 = FeatureFilter('QUALITY', 50, True)
        settings.addSpotFilter(filter1)
        print(settings.detectorSettings)
        
        # Tracker
        settings.trackerFactory = SparseLAPTrackerFactory()
        settings.trackerSettings = settings.trackerFactory.getDefaultSettings()
        for parameter, value in tsettings.items():
            #settings.trackerSettings.update({parameter:value})
            settings.trackerSettings[parameter] = value
        
        # Execute Tracking
        trackmate = TrackMate(model, settings)
        ok = trackmate.checkInput()
        if not ok:
            sys.exit(str(trackmate.getErrorMessage()))
        ok = trackmate.process()
        if not ok:
            sys.exit(str(trackmate.getErrorMessage()))
        selectionModel = SelectionModel(model)
        
        # Display
        ds = DisplaySettingsIO.readUserDefault()
        displayer = HyperStackDisplayer(model, selectionModel, imp, ds)
        displayer.render()
        displayer.refresh()
        trackscheme = TrackScheme(model, selectionModel, ds)
        #trackscheme.render()
        
        # Save Data
        outFile = Path(out+image+"_exportModel.xml")
        writer = TmXmlWriter(outFile)
        writer.appendModel(model)
        writer.appendSettings(settings)
        writer.writeToFile()
        csvFileTracks = Path(out+image+"_exportTracks.csv")
        trackTableView = TrackTableView(model, selectionModel, ds)
        trackTableView.getSpotTable().exportToCsv(csvFileTracks)

In [15]:
ij.py.run_macro('close("*")')

<java object 'org.scijava.script.ScriptModule'>

### Joining of trackmate output and ROI coordinates from fiji

In [19]:
from math import isnan


def xref_locations(first, second, first_x='POSITION_X', first_y='POSITION_Y', first_z='POSITION_Z',
                   second_x='X', second_y='Y', second_z='Slice',
                   max_dist=20, verbose=False):
    pairwise_elements = pandas.DataFrame()
    first_measurements = pandas.read_csv(first)
    first_measurements = first_measurements.drop([0,1,2])
    second_measurements = pandas.read_csv(second)
    first_gdf = geopandas.GeoDataFrame(
        first_measurements,
        geometry=geopandas.points_from_xy(first_measurements[first_x],
                                          first_measurements[first_y],
                                          first_measurements[first_z]))
    second_gdf = geopandas.GeoDataFrame(
        second_measurements,
        geometry=geopandas.points_from_xy(second_measurements[second_x],
                                          second_measurements[second_y],
                                          second_measurements[second_z]))
    ti_rows = first_gdf.shape[0]
    tj_rows = second_gdf.shape[0]
    for ti_row in range(0, ti_rows):
        if verbose:
            print(f"On row: {ti_row}")
        ti_element = first_gdf.iloc[[ti_row, ]]
        # Get the frame of the current element
        ti_frame = int(ti_element['FRAME'].values[0])  # Convert to integer values in 'FRAME' column
        # Filter second_gdf to only include elements with the same frame
        same_frame_elements = second_gdf[second_gdf['Frame'] == (ti_frame + 1)]
        
        titj = geopandas.sjoin_nearest(ti_element, same_frame_elements,
                                       distance_col="pairwise_dist",
                                       max_distance=max_dist)
        chosen_closest_dist = titj.pairwise_dist.min()
        if (isnan(chosen_closest_dist)):
            print(f"This element has no neighbor within {max_dist}.")
        else:
            chosen_closest_cell = titj.pairwise_dist == chosen_closest_dist
            chosen_closest_row = titj[chosen_closest_cell]
            pairwise_tmp = pandas.concat([pairwise_elements, chosen_closest_row])
            pairwise_elements = pairwise_tmp
    return pairwise_elements


In [20]:
import matplotlib.pyplot as plt
file_pattern = os.path.join(bact_path, "*.tif")
file_list = glob.glob(file_pattern)

for file_path in file_list:
    filename = os.path.basename(file_path)
    filename2 = os.path.splitext(filename)[0]
    first = Path(f"{bact_path}/Output/{filename}_exportTracks.csv")
    second = Path(f"{bact_path}/{filename2}.csv")
    #first = '/home/saka/Documents/Lab_stuff/confocal/exp2/tracks_from_script.csv'
    #second = '/home/saka/Documents/Lab_stuff/confocal/exp2/results_bacteria.csv'
    pairwise = xref_locations(first, second, 
                          first_x='POSITION_X', 
                          first_y='POSITION_Y', 
                          first_z='POSITION_Z', 
                          second_x='X', 
                          second_y='Y', 
                          second_z='Slice', 
                          verbose=False)
    #pairwise.head()
    grouped = pairwise.groupby('TRACK_ID')['Unnamed: 0'].apply(list).reset_index()
    grouped.rename(columns={'Unnamed: 0': 'object_ID_list'}, inplace=True)
    final_csv = Path(f"{directory_path}/{filename2}_grouped.csv")
    grouped.to_csv(final_csv)

In [ ]:
print(len(file_list))

### Quality control of bacterial tracking

In [ ]:
directory_path = '/home/saka/Documents/Lab_stuff/confocal/test_script_trackmate/'
rm = ij.RoiManager.getRoiManager()

In [21]:
import random
file_pattern = os.path.join(bact_path, "*.tif")
file_list = glob.glob(file_pattern)
colors = ["blue", "cyan", "green", "magenta", "orange", "red", "yellow"]
for file_path in file_list: # cycle through images, open them and open the matching "grouped' csv file
    imp = ij.IJ.openImage(file_path)
    ij.ui().show(imp)
    f_name = os.path.basename(file_path)
    f_name = os.path.splitext(f_name)[0]
    input_csv = Path(f"{directory_path}/{f_name}_grouped.csv")
    df = pandas.read_csv(input_csv)
    pouet = df['object_ID_list']
    input_ROI = Path(f"{bact_path}/{f_name}.zip")
    rm.open(f"{input_ROI}")
    for i in range(len(pouet)):
        single_row = df.iloc[i]
        random_color = random.choice(colors)
        row_cellids = single_row.object_ID_list
        row_cleaned = row_cellids.strip('[').strip(']')
        row_array = row_cleaned.split(', ')
        for cell in row_array:
            cell_index = int(cell)
            roi = rm.select(cell_index)
            overlay_command = f"Overlay.addSelection('{random_color}',2);"
            ij.py.run_macro(overlay_command)
    ij.py.run_macro("setMinAndMax(0, 50);")
    ij.py.run_macro("run('Flatten', 'stack');")
    method = 'max all'
    z_projector_result = ZProjector.run(imp, method)
    z_collapsed_image = ij.py.from_java(z_projector_result)
    z_collapsed_dataset = ij.py.to_dataset(z_projector_result)
    result_path = os.path.splitext(file_path)[0] + "_overlay.tif"
    ij.io().save(z_collapsed_dataset, result_path)
    ij.py.run_macro("close();")
    ij.py.run_macro("roiManager('Select All');")
    rm.runCommand("Delete")
    

### Quantification on single MCV

In [23]:
channel_path = Path(f"{directory_path}/other")
file_pattern = os.path.join(channel_path, "*.tif")
file_list = glob.glob(file_pattern)

In [ ]:
len_list = int(len(file_list)/2)
print(len_list)

In [ ]:
print(file_list)

In [12]:
file_path = '/home/saka/Documents/Lab_stuff/confocal/exp1ATG7_scene3/other/LC3_single_cell_13.tif'
prefix_lc3 = 'LC3_'
basename = os.path.basename(file_path)
f_name = os.path.splitext(basename)[0]
f_name2 = f_name.strip(prefix_lc3)
if basename.startswith(prefix_lc3):
    print(basename)

In [24]:
#set measurement 
set_string = f'Set Measurements...'
measure_string = f'mean stack redirect=None decimal=2'
ij.IJ.run(set_string, measure_string)
cois = ['LC3', 'LV']
prefix_bact = 'bact'

# preparation of variables for file name call automation
for file_path in file_list:
    f_name = os.path.basename(file_path)
    basename = os.path.splitext(f_name)[0]

    if basename.startswith(cois[0]):
        corename = basename.split("_", maxsplit=1)[1]
        input_csv = Path(f"{directory_path}/{corename}_grouped.csv")
        input_roi = Path(f"{directory_path}/bact/{corename}.zip")
        df0 = pandas.read_csv(input_csv)
        rm.open(f"{input_roi}")

        for channel in cois:
            c_path = Path(f"{directory_path}/other/{channel}_{corename}.tif").as_posix()
            image_c = ij.io().open(c_path)
            ij.ui().show(image_c)
            for n in range(len(df0)):
                single_row = df0['object_ID_list'][n] #testing.iloc[10] #row number -2
                row_cleaned = single_row.strip('[').strip(']')
                terms = row_cleaned.split(", ")
                nums = []
                for term in terms:
                    nums.append(int(term))
                nums.sort()
                for bact in nums:
                    roi = rm.select(bact)
                    ij.IJ.run('Measure')
            output = Path(f"{directory_path}/{channel}_{corename}.csv").as_posix()
            saving = ij.IJ.saveAs("Results", output)
            ij.IJ.run("Clear Results")
            ij.IJ.selectWindow(f"{channel}_{corename}.tif")
            ij.IJ.run('Close')

In [ ]:
file_pattern = os.path.join(directory_path, "*.tif.csv")
file_list = glob.glob(file_pattern)
len_list = int(len(file_list)/2)
print(len_list)

In [1]:
cell_list = []
for filename in os.listdir(directory_path+"/measurement"):
    if filename.startswith(cois[0]):
        basename = filename[len(cois[0])+1:]
        corename = os.path.splitext(basename)[0]
        cell_list.append(corename)

for cell in cell_list:
    input_csv_lc3 = Path(f"{directory_path}/measurement/LC3_{cell}.csv")
    input_csv_lv = Path(f"{directory_path}/measurement/LV_{cell}.csv")
    
    #first marker csv
    df1 = pandas.read_csv(input_csv_lc3)
    df1.rename(columns={'Mean': 'Mean_LC3'}, inplace=True)
    df1 = df1.drop(columns = [df1.columns[0],df1.columns[2]], axis =1)

    #second marker csv
    df2 = pandas.read_csv(input_csv_lv)
    df2.rename(columns={'Mean': 'Mean_LV'}, inplace=True)
    df2 = df2.drop(columns =[df2.columns[0],df2.columns[2], df2.columns[3]], axis =1)

    #create new merged dataframe:
    final_results = pandas.concat([df1, df2], axis = 1)
    final_results = final_results.iloc[:, [1, 0, 2]]

    # add a column for track_ids (based on grouped.csv)
    input_csv = Path(f"{directory_path}/grouped/{cell}_grouped.csv")
    df0 = pandas.read_csv(input_csv)
    track_ids = []
    for index, row in df0.iterrows():
        cellnums = row['object_ID_list'].strip('[').strip(']').split(", ")
        for cellnum in cellnums:
            track_ids.append(index)
    final_results['track_id'] = track_ids
    
    out = directory_path +"/final_output/"
    if not os.path.exists(out):
        os.makedirs(out)
    output_path = Path(f"{out}/{cell}_final_result.csv")
    final_results.to_csv(output_path)

NameError: name 'os' is not defined